In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
df_train = pd.read_csv('[new] yancheng_train_20171226.csv')

In [3]:
df_train.columns

Index(['sale_date', 'class_id', 'sale_quantity', 'brand_id', 'compartment',
       'type_id', 'level_id', 'department_id', 'TR', 'gearbox_type',
       'displacement', 'if_charging', 'price_level', 'price', 'driven_type_id',
       'fuel_type_id', 'newenergy_type_id', 'emission_standards_id',
       'if_MPV_id', 'if_luxurious_id', 'power', 'cylinder_number',
       'engine_torque', 'car_length', 'car_width', 'car_height',
       'total_quality', 'equipment_quality', 'rated_passenger', 'wheelbase',
       'front_track', 'rear_track'],
      dtype='object')

In [4]:
# descriptive statistics summary
df_train["sale_quantity"].describe()

count    20157.00000
mean       126.31205
std        231.13742
min          5.00000
25%         16.00000
50%         47.00000
75%        142.00000
max       6284.00000
Name: sale_quantity, dtype: float64

In [9]:
df_test = pd.read_csv('yancheng_testA_20171225.csv')

In [10]:
df_test.head()

,predict_date,class_id,predict_quantity
0,201711,103507,NaN
1,201711,124140,NaN
2,201711,125403,NaN
3,201711,136916,NaN
4,201711,169673,NaN


# the follows is show the each class_id the relationship between sale_date and sale_quantity

In [11]:
df_train_feature1=pd.concat([df_train['class_id'],df_train['sale_date'],df_train['sale_quantity']],axis=1)

In [27]:
class_id=df_train_feature2['class_id'].unique()

In [28]:
class_id

array([103507, 124140, 125403, 136916, 169673, 175962, 178529, 186250,
       194201, 194450, 198427, 206765, 209945, 219195, 221795, 245609,
       248352, 249875, 250658, 265980, 270690, 281301, 281792, 289386,
       289403, 290854, 291086, 291514, 302513, 304458, 308913, 321683,
       340175, 347384, 348641, 349023, 350259, 354068, 356986, 359378,
       360648, 376193, 378356, 378536, 379265, 379876, 384924, 392593,
       395234, 401052, 409854, 412240, 416749, 417803, 425432, 436105,
       437063, 437598, 453752, 468392, 472603, 475895, 481950, 482497,
       492952, 496459, 501564, 510309, 516750, 526401, 527765, 527809,
       540712, 559132, 560265, 569154, 576298, 580470, 580634, 587678,
       591790, 597927, 603972, 612523, 614059, 619401, 621073, 628041,
       649213, 651347, 651782, 653436, 654134, 671482, 682651, 683364,
       687270, 692703, 693436, 713651, 714150, 714152, 714860, 725137,
       732758, 735971, 736094, 739296, 741152, 743957, 745137, 750340,
      

In [16]:
data_s=[]
for i in df_train_feature1['sale_date']:
    i = str(i)
    s=i[2:4]+'-'+i[-2:]
    data_s.append(s)

In [17]:
df_train_feature1['date_s']=data_s

In [23]:
df_train_feature2=df_train_feature1.groupby(['class_id','sale_date']).agg('sum').reset_index()

In [24]:
df_train_feature2

,class_id,sale_date,sale_quantity
0,103507,201503,58
1,103507,201504,232
2,103507,201505,226
3,103507,201506,286
4,103507,201507,297
5,103507,201508,355
6,103507,201509,442
7,103507,201510,1050
8,103507,201511,481
9,103507,201512,1433


### 这里加date_num本来用意打算用xgboost预测的时候建模型,最后打算直接取8、9、10三个月平均值

In [42]:
date_num=[]
for i in df_train_feature2['sale_date']:
    s=((i-201000)//100)*12+((i-201000)%100)
    date_num.append(s)

In [43]:
df_train_feature2['date_num']=date_num

In [44]:
df_train_feature2

,class_id,sale_date,sale_quantity,date_num
0,103507,201503,58,63
1,103507,201504,232,64
2,103507,201505,226,65
3,103507,201506,286,66
4,103507,201507,297,67
5,103507,201508,355,68
6,103507,201509,442,69
7,103507,201510,1050,70
8,103507,201511,481,71
9,103507,201512,1433,72


### 对于小于3个月的数据，11月份的数据保持和10月份一致

In [50]:
class_num_dataset=[]
for i in class_id:
    class_id_num=df_train_feature2[df_train_feature2['class_id']==i]
    class_num=len(class_id_num)
    if class_num < 3:#只在2017年产生的数据
        class_num_dataset.append(i)
class_num_dataset

[194201, 359378, 376193, 653436]

In [51]:
for i in class_num_dataset:
    ss=df_train_feature2[df_train_feature2['class_id']==i]
    print(ss)

     class_id  sale_date  sale_quantity  date_num
369    194201     201709            239        93
370    194201     201710            378        94
      class_id  sale_date  sale_quantity  date_num
1798    359378     201709             16        93
1799    359378     201710             64        94
      class_id  sale_date  sale_quantity  date_num
1846    376193     201709            133        93
1847    376193     201710             90        94
      class_id  sale_date  sale_quantity  date_num
3908    653436     201709            122        93
3909    653436     201710            171        94


In [83]:
sale_quantity_dict={}
ss_08=df_train_feature2[df_train_feature2['sale_date']==201708]
ss_09=df_train_feature2[df_train_feature2['sale_date']==201709]
ss_10=df_train_feature2[df_train_feature2['sale_date']==201710]
for i in class_id:
    if i in class_num_dataset:
        continue
    ss_08_quantity=ss_08[df_train_feature2['class_id']==i]['sale_quantity']
    ss_09_quantity=ss_09[df_train_feature2['class_id']==i]['sale_quantity']
    ss_10_quantity=ss_10[df_train_feature2['class_id']==i]['sale_quantity']
    sale_quantity_dict[i]=(int(ss_08_quantity)+int(ss_09_quantity)+int(ss_10_quantity))//3
    

In [85]:
for i in class_num_dataset:
    sale_quantity_dict[i]=int(ss_10[df_train_feature2['class_id']==i]['sale_quantity'])

In [86]:
len(sale_quantity_dict)

140

In [87]:
for i in class_id:
    df_test.loc[df_test['class_id']==i,'predict_quantity']=sale_quantity_dict[i]

## 验证一下


In [88]:
 df_test.loc[df_test['class_id']==861459,'predict_quantity']

122    229.0
Name: predict_quantity, dtype: float64

In [89]:
sale_quantity_dict[861459]

229

# save

In [91]:
df_test.to_csv("../../raw/Woody/result_woody20180206_1.csv", index=False)